In [2]:
from instrument import instrument #utility to help visualize recursive calls (on stderr)

In [3]:
instrument.SHOW_CALL = True
instrument.SHOW_RET = True

# Recursive Patterns
## Let's start with some simple functions that recurse on lists...
### Walk the list to find the first value satisfying function `f`

In [9]:
@instrument
def walk_list(L, f):
    """Walk a list -- in a recursive style. Note that this is done as a
    stepping stone toward other recursive functions, and so does not
    use easier/direct built-in list functions.
    
    In this first version -- walk the list just to find/return the
    FIRST item that satisfies some condition, where f(item) is true.
    
    >>> walk_list([1, 2, 3], lambda x: x > 2)
    3
    """
    if L == []:
        return None
    
    if f(L[0]):
        return L[0]
    
    else:
        return walk_list(L[1:], f)

In [10]:
walk_list([1, 2, 3], lambda x: x > 2)
walk_list([1, 2], lambda x: x > 2)

call to walk_list: [1, 2, 3], <function <lambda> at 0x0000013CB890D9D8>
   call to walk_list: [2, 3], <function <lambda> at 0x0000013CB890D9D8>
      call to walk_list: [3], <function <lambda> at 0x0000013CB890D9D8>
      walk_list returns: 3
   walk_list returns: 3
walk_list returns: 3
call to walk_list: [1, 2], <function <lambda> at 0x0000013CB890D9D8>
   call to walk_list: [2], <function <lambda> at 0x0000013CB890D9D8>
      call to walk_list: [], <function <lambda> at 0x0000013CB890D9D8>
      walk_list returns: None
   walk_list returns: None
walk_list returns: None


### Walk a list, but now returning a <em>list</em> of items that satisfy `f` 

In [13]:
@instrument
def walk_list_filter(L, f):
    """ Walk a list, returning a list of items that satisfy the
    condition f.

    >>> walk_list_filter1([1, 2, 3], lambda x: x % 2 == 1) #odd only
    [1, 3]
    """
    if L == []:
        return []
    
    if f(L[0]):
        return [L[0]] + walk_list_filter(L[1:], f)
    
    else:
        return walk_list_filter(L[1:], f)

In [14]:
walk_list_filter([1, 2, 3], lambda x: x % 2 == 1)

call to walk_list_filter: [1, 2, 3], <function <lambda> at 0x0000013CB890DEA0>
   call to walk_list_filter: [2, 3], <function <lambda> at 0x0000013CB890DEA0>
      call to walk_list_filter: [3], <function <lambda> at 0x0000013CB890DEA0>
         call to walk_list_filter: [], <function <lambda> at 0x0000013CB890DEA0>
         walk_list_filter returns: []
      walk_list_filter returns: [3]
   walk_list_filter returns: [3]
walk_list_filter returns: [1, 3]


[1, 3]

## Now consider some functions that recurse on trees...
We want to extend the basic idea of recursive walkers and builders for lists, now to trees. We'll see the same patterns at work, but now often with more base cases and/or more recursive branch cases.

For these examples, we need a simple tree structure. Here we'll represent a node in a tree as a list with the first element being the node value, and the rest of the list being the children nodes.
That is to say, our tree structure is a simple nested list structure.
<center><img src="images/tree1.png" alt="tree [13, [7], [8, [99], [16], [42]]]" width=150></center>

In [26]:
tree1 = [13,
            [7],
            [8, 
                [99],
                [16, 
                    [77]
                ],
                [42]
            ]
        ]
tree1

[13, [7], [8, [99], [16, [77]], [42]]]

In [ ]:
@instrument
def depth_tree(tree):
    """ Walk a tree, returning the depth of the tree
    >>> depth_tree([13, [7], [8, [99], [16, [77]], [42]]])
    4
    """
    pass

In [ ]:
depth_tree([13, [7], [8, [99], [16, [77]], [42]]])

In [ ]:
@instrument
def tree_max(tree):
    """Walk a tree, returning the maximum value in the (assumed non-empty) tree.
    >>> tree_max([13, [7], [8, [99], [16, [77]], [42]]])
    99    
    """
    pass

In [ ]:
tree_max(tree1)

Notice that the recursion structure is exactly the same in both cases?  We could generalize to something like a `walk_tree` that took a tree *and* a function `f` (and perhaps some other base case values), and did that operation at each step. We'll leave that as an exercise for the reader. 

## How might we flatten a tree?

A lot of time we want to process data structures we into a different forms. Let's write a function that can take a tree, or nested list, and return the elements within one non-nested list. Lets assume we always see the elements to the left first so we return them in the same order they appear in the input.

In [30]:
@instrument
def flatten(L):
    """ Flatten a nested list, returning all the 
    elements in the same order they appear within one list.

    >>> flatten([1, [[2, 4], 3]])
    [1, 2, 4, 3]
    """
    new_L = []
    for el in L:
        if isinstance(el, list):
            new_L.extend(flatten(el))
        else:
            new_L.append(el)
    return new_L

In [31]:
flatten([1, [[2, 4], 3]])

call to flatten: [1, [[2, 4], 3]]
   call to flatten: [[2, 4], 3]
      call to flatten: [2, 4]
      flatten returns: [2, 4]
   flatten returns: [2, 4, 3]
flatten returns: [1, 2, 4, 3]


[1, 2, 4, 3]

## Finally, consider some functions that recurse on directed graphs...
For this, we need a more sophisticated structure, since a node may be referenced from more than one other node. We'll represent a directed graph (also known as a "digraph") as a dictionary with node names as keys, and associated with the key is a list holding the node value and a list of children node names. The special name 'root' is the root of the graph.

<center><img src="images/graph1.png" alt="graph1" width=300></center>


Moreover, graphs may also contain cycles! E.g.,

<center><img src="images/graph2.png" alt="graph with cycle" width=300></center>

How do we avoid infinite recursion?

In [ ]:
graph1 = {'root': [13, ['A', 'B']],
          'A': [77, ['B', 'C']],
          'B': [88, []],
          'C': [-32, ['D']],
          'D': [42, []],
          'E': [242, []]}

In [ ]:
graph2 = {'root': [13, ['A', 'B']],
          'A': [77, ['B', 'C']],
          'B': [88, []],
          'C': [-32, ['D']],
          'D': [42, ['A']],  # D --> A (cycle)
          'E': [242, []]} 

In [ ]:
@instrument
def graph_max(graph):
    """Walk a graph, returning the maximum value reachable from the root 
    in a (non-empty) graph. However, there might be cycles, so need to 
    be careful not to get stuck in them!
    """

    @instrument
    def node_max(node_name):
        pass

    return node_max('root')

In [ ]:
instrument.SHOW_CALL = True
instrument.SHOW_RET = True

In [ ]:
graph_max(graph1)

In [ ]:
graph_max(graph2)